# Testing Calibration V2Worksheet

## 1. Setup
This section load python objects and variables please check the ```cam``` variable

### 1.1 Load python objects
Loads ```scope``` and ```lamp``` python objects for reading sekonic and communication with the lamp

In [1]:
cam = "http://192.168.2.224/video_feed"
sharedFolder = r"C:\Users\Virtual Production 2\Documents\ApolloLogs\\"
comPort = 'COM6'
brightnessString = "self.slider_h.value = (240, 399)\nself.slider_v.value = (378, 436)\nself.minus_comp.value = False\nself.minus_brightness.value = '150'\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.')]) \n"
duvString = "self.slider_h.value = (261, 450)\nself.slider_v.value = (296, 354)\nself.minus_comp.value = True\nself.minus_brightness.value = '150'\nself.offset_x.value = -85\nself.offset_y.value = 3\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('20.0', '0,0'), ('0.6', '0.0'), ('0.8', '0.0')]) \n"
kelvinString = "self.slider_h.value = (265, 423)\nself.slider_v.value = (214, 272)\nself.minus_comp.value = False\nself.minus_brightness.value = '150'\nself.offset_x.value = 0\nself.offset_y.value = 0\nself.rota.value = '90'\nself.flip.value = False\nself.flop.value = False\nself.replacement_list.value =str([('o', '0'), ('O', '0'), (',', '.'), ('G', '6')]) \n"
data = None

%run calibration_setup_v2.ipynb

pygame 2.1.2 (SDL 2.0.18, Python 3.10.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


#### [Optional] Find COM port

In [ ]:
# Get COM port list
import serial.tools.list_ports

com_ports = list(serial.tools.list_ports.comports())

for port, desc, hwid in com_ports:
    print(str(port)+" "+str(desc))

### 1.2 Connect to lamp
Lamp needs to be closed before it can be opened again.
If you dont know the serial port where the lamp is connected use the Find COM port section
1st You need to setup sekonic and feedback camera (the areas where the data will be read, 1st -green value is set up so it would be possibel to get dUV - sign location), then close the lamp and start again with the scope objects

In [3]:
lamp1.close()

In [4]:
lamp1 = apollo_device(ComPort=comPort)
lamp1.getId()
lamp1.macId = lamp1.macId.replace(":","")
lamp1.macId

'CCDBA78AE200'

In [5]:
lamp1.color = [200,50,50,500,0]
lamp1.update()

### 1.3 Set Sekonic camera areas
Set the area where to "read" the Tcp, dUV and lux variables
Once the object is initalized tune the paramters and test by ```update()```
To save the paramteres use the Setup below to generate the ```saved_string``` and copy and paste it into funtion paramter and save the workbook

In [6]:
Brightness = scope(saved_string=brightnessString)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [7]:
Brightness.update()

5810

In [8]:
dUV = scope(saved_string=duvString)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [9]:
dUV.update()

-0.0333

In [10]:
Kelvin = scope(saved_string=kelvinString)

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


Label(value='Out:')

Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x87\x00\x00\x002\x08\x02\x00\x00\x00\xf9\x82_\x9…

Output()

In [11]:
Kelvin.update()

4593

#### Setup

In [ ]:
Brightness.save_string()

In [ ]:
dUV.save_string()

In [ ]:
Kelvin.save_string()

# Test

In [15]:
lamp1.close()
time.sleep(0.5)

In [13]:
lamp1 = apollo_device(ComPort=comPort,SekBrightnes=Brightness, SekDuv=dUV, SekKelvin=Kelvin)
lamp1.getId()

# Disable DMX
cmd = "De0 \n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)

lamp1.macId = lamp1.macId.replace(":","")
lamp1.macId

{"dmx_enabled":0,"Timestamp":1703108063758063900}


'CCDBA78AE200'

In [ ]:
%%time

startCalib = 3
endCalib = 5
readCalibration = True
writeCalibration = True
allowedKelvinDifference = 20
allowedDuvDifference = 0.0002
maxTries = 30

data = None

# try N times to get the dUV & Kelvin difference low enough
for kelvin in range(startCalib,endCalib+1):
    print(f"starting kelvin range {KelvinRangeIds[kelvin]}\n")
    done = False
    for i in range(maxTries):
        if done:
            break  # Exit the inner loop when calibration is successful
        measured = calRange(KelvinTemp=kelvin, readCalibration=readCalibration, writeCalibration=writeCalibration)
        if (abs(measured[1]) < allowedDuvDifference) and (abs(measured[0]-KelvinRangeIds[kelvin]) < allowedKelvinDifference):
            done = True
    if done:
        # Record final results for proofing
        new_data_line = {
            'wb': [KelvinRangeIds[kelvin]],
            'brightness_point': [8],
            'kelvin': [measured[0]],
            'duv': [measured[1]],
            'lux': [measured[2]]
        }
        new_data_line_df = pd.DataFrame(new_data_line)
        print("Debug new_data_line: " + str(new_data_line))
        if data is None:
            data = new_data_line_df
        else:
            data = pd.concat([data, new_data_line_df], ignore_index=True)
        

In [14]:
%%time

startCalib = 2     # 0 0-5 [2800,3200,4800,5600,7800,10000]
endCalib = 5       # 5 0-5 [2800,3200,4800,5600,7800,10000]
brightnessSart = 7 # 7 1-7 
readCalibration = True
writeCalibration = True
maxTries = 30
allowedRanges = [
    [0,0.0000,0],
    [1,0.0050,500],
    [2,0.0020,300],
    [3,0.0005,200],
    [4,0.0005,100],
    [5,0.0005,100],
    [6,0.0005,100],
    [7,0.0005,100]]
       

def calKelvinRange(KelvinTemp = 2, readCalibration = True, writeCalibration = True):
    global calibration_points, data, mergedLux;

    if readCalibration:
        print("Loading calibartion from lamp\n")
        lamp1.send(("Cr"+str(KelvinTemp)+" "+str(brightnessSart+1)+" \n").encode('utf-8'))
        lamp1.read()
        jsonString = "{"+lamp1.lastValAsJsonString[1:-1]+"}"
        jsonObject = eval(jsonString)

        print("loaded values: "+str(jsonObject)+"\n")
        # If below 1 do not read the value, Us the last good value (factory setting or last lamp)
        if jsonObject['r'] > 0:
            calibration_points[KelvinTemp][brightnessSart + 1][2] = jsonObject['r']
        if jsonObject['g'] > 0:
            calibration_points[KelvinTemp][brightnessSart + 1][3] = jsonObject['g']
        if jsonObject['b'] > 0:
            calibration_points[KelvinTemp][brightnessSart + 1][4] = jsonObject['b']
        if jsonObject['w'] > 0:
            calibration_points[KelvinTemp][brightnessSart + 1][5] = jsonObject['w']

        # The answer is 2 line, so to clear the buffer
        lamp1.read()

    for lx in reversed(range(1,7+1)):
        # on first passthrough skip the lx above the brightness start, later kelvin values will be started from 7 agaian
        if KelvinTemp == startCalib and lx > brightnessSart:
            break
            
        ratio = mergedLux[KelvinTemp][lx+1] / mergedLux[KelvinTemp][lx]
        print(str(ratio)+"\n")
        split = [round(x/ratio) for x in calibration_points[KelvinTemp][lx+1]]
        calibration_points[KelvinTemp][lx][2] = split[2]
        calibration_points[KelvinTemp][lx][3] = split[3]
        calibration_points[KelvinTemp][lx][4] = split[4]
        calibration_points[KelvinTemp][lx][5] = split[5]
        print(str(calibration_points[KelvinTemp][lx])+"\n")
        done = False
        
        # try N times to get the dUV & Kelvin difference low enough
        for i in range(maxTries):
            if done:
                break
            measured = calRange(KelvinTemp=KelvinTemp,BrightnessV=lx,readCalibration=False,writeCalibration=writeCalibration)
            if (abs(measured[1]) < allowedRanges[lx][1]) and (abs(measured[0]-KelvinRangeIds[kelvin]) < allowedRanges[lx][2]):
                done = True
        
        if done:
            # record final results for proofing
            newDataLine = {
                'wb': [KelvinRangeIds[KelvinTemp]],
                'brightness_point': [lx],
                'kelvin': [measured[0]],
                'duv': [measured[1]],
                'lux': [measured[2]]
            }
            newDataLineDf = pd.DataFrame(newDataLine)
            print("Debug newDataLine: "+str(newDataLine))
            if data is None:
                data = newDataLineDf
            else:
                data = pd.concat([data, newDataLineDf], ignore_index=True)

#
for kelvin in range(startCalib,endCalib+1):
    print(f"starting kelvin range {KelvinRangeIds[kelvin]}\n")
    calKelvinRange(KelvinTemp=kelvin,readCalibration=readCalibration,writeCalibration=writeCalibration)  

starting kelvin range 4800

Loading calibartion from lamp

loaded values: {'wb': 2, 'brightness': 8, 'r': 574, 'g': 537, 'b': 21, 'w': 1413, 'Timestamp': 1703108069837682200}

2.0

[4800, 7, 287, 268, 10, 706]

Adjusting lamp brightness to 10000lx in 4800K with [1, 1, 0, 3]/[11, 10, 0, 26] steps
current brightness: 20700, ideal 10000, diff -10700 : [287, 268, 10, 706, 0] / [9305, 268, 10, 706, 0]


KeyboardInterrupt: 

In [ ]:
calibration_points[2]

In [ ]:
import datetime

# Save Log
file_path = sharedFolder+"Apollo"+str(lamp1.macId)+"_"+datetime.datetime.now().strftime("%d%m%Y_%H%M%S")+"caliblog.csv"
data.to_csv(file_path, index=False)
print("Saved to: "+file_path)

# Save Calibration
with open(sharedFolder+"Apollo"+str(lamp1.macId)+"_"+datetime.datetime.now().strftime("%d%m%Y_%H%M%S")+"calib.py", "w") as file:
    file.write(str(calibration_points))


In [ ]:
from mitosheet import *; register_analysis("id-uhsskvzpjn");

# Filtered duv
data = data[data['duv'] > 0.001]

# Formatted dataframes. View these styling objects to see the formatted dataframe
data_styler = data.style\
    .format("{:.4f}", subset=['duv'])


In [ ]:
playSucess()

In [ ]:
cmd = "Cr3 8\n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)
lamp1.read()
print(lamp1.lastValAsJsonString)

In [ ]:
lamp1.read()

In [ ]:

cmd = "Cw4 7 15000 100 30 30 21 \n"
lamp1.send(cmd.encode('utf-8'))
lamp1.read()
print(lamp1.lastValAsJsonString)
